# Inicializar la Dimensión hora
importaciones


In [7]:
from sqlalchemy import create_engine
import pandas as pd
import yaml

Crear un rango de horas para un solo día, con un paso de una hora

In [3]:
horas = pd.date_range(start='00:00', end='23:59', freq='1min', inclusive='left')

Crear el DataFrame de la dimensión hora

In [4]:
dim_hora = pd.DataFrame({
    "hora_completa": horas.time  # Solo la parte de la hora (HH:MM:SS)
})

Añadir columnas derivadas

In [5]:
dim_hora["hora_id"] = dim_hora["hora_completa"].apply(lambda x: int(x.strftime('%H%M')))  # ID de la hora en formato HHMM
dim_hora["hora"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%H'))  # Solo la hora (00, 01, ..., 23)
dim_hora["minuto"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%M'))  # Solo el minuto (00, 01, ..., 59)
dim_hora["descripcion"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%H:%M'))  # Descripción en formato HH:MM

# Ver columnas del DataFrame

In [6]:
dim_hora.head()

,hora_completa,hora_id,hora,minuto,descripcion
0,00:00:00,0,00,00,00:00
1,00:01:00,1,00,01,00:01
2,00:02:00,2,00,02,00:02
3,00:03:00,3,00,03,00:03
4,00:04:00,4,00,04,00:04


# Conexión a la Base de Datos

In [8]:
# Cargar configuración de conexión desde el archivo YAML
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Crear la dimensión hora (similar al ejemplo anterior)

In [9]:
horas = pd.date_range(start='00:00', end='23:59', freq='1min', inclusive='left')

# Crear el DataFrame de la dimensión hora

In [10]:
dim_hora = pd.DataFrame({
    "hora_completa": horas.time  # Solo la parte de la hora (HH:MM:SS)
})

# Añadir columnas derivadas

In [11]:
dim_hora["hora_id"] = dim_hora["hora_completa"].apply(lambda x: int(x.strftime('%H%M')))  # ID de la hora en formato HHMM
dim_hora["hora"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%H'))  # Solo la hora (00, 01, ..., 23)
dim_hora["minuto"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%M'))  # Solo el minuto (00, 01, ..., 59)
dim_hora["descripcion"] = dim_hora["hora_completa"].apply(lambda x: x.strftime('%H:%M'))  # Descripción en formato HH:MM


# Cargar la dimensión hora en la base de datos ETL

In [12]:
dim_hora.to_sql('dim_hora', etl_conn, if_exists='replace', index_label='key_dim_hora')

439